In [1]:
from py2neo import Graph
import pandas as pd

In [2]:
# connect to the graph -> it should be running already
graph = Graph(scheme="bolt", host="localhost", port=7687, auth=('neo4j', '12345'))

# connect to test database, only accessible inside ipf network
# pw = ''
# graph = Graph("bolt://neo4j.loc.ipfdd.de/newsocket",auth=('neo4j',pw))


In [3]:
import neo2BFMPolymer as neo2BFM

In [4]:
# simulation project function
def insertCodmucSimRunInGraph(myNeo4Polymer, simProject, simRunName, filepath, generation, spacerLength, graftedChainLength):
    '''insert all information of a codendrimer simulation Run (Codmuc) to the database'''
    myNeo4Polymer.addSimulationRunToSimulationProject(currentSimRun,currentProject)
    myNeo4Polymer.addBFMFileToDatabase(currentSimRun,filepath)
    myNeo4Polymer.addDendrimerGenerationToSimulationRun(currentSimRun,generation)
    myNeo4Polymer.addDendrimerSpacerLengthToSimulationRun(currentSimRun,spacer)
    myNeo4Polymer.addGraftedChainLengthToSimulationRun(currentSimRun,graftedChainLength)

# this ALWAYS deletes the full graph! Use this function with care
graph.delete_all()

# setup instacne of neo4j - BFM file interface
myNeoPolymers = neo2BFM.neo2BFMPolymer(graph)

### -- preparation of project -- ###
# add simulation project:
currentProject = "Linear-Dendritic Copolymer Solutions"
myNeoPolymers.addNewSimulationProject(currentProject)
# add grafted chain as polymer:
myNeoPolymers.addNewPolymer("Grafted Chain")
# add dendrimer as polymer:
myNeoPolymers.addNewPolymer("Dendrimer")
# connect Dendrimers:
myNeoPolymers.connectSimulationToPolymer("Dendrimer", currentProject)
# connect Grafted Chain:
myNeoPolymers.connectSimulationToPolymer("Grafted Chain", currentProject)


### -- read in all information of simRun 1 -- ###
generation = 3
spacer = 4
numMolecules = 27
graftedChainLength = 4
currentSimRun = "codmuc_micelle_g{}_s{}_m{}_l{}_e0.8_b128".format(generation,spacer,numMolecules,graftedChainLength)
filepath = "bfm_files/{}_1425000000_lastconfig.bfm".format(currentSimRun)
# use simulation project function:
insertCodmucSimRunInGraph(myNeoPolymers, currentProject, currentSimRun, filepath, generation, spacer, graftedChainLength)
# add radius of gyration results
filepath = "bfm_files/{}_rg.dat".format(currentSimRun)
myNeoPolymers.addCODMUCRgTensorFileToDatabase(currentSimRun,filepath)

### -- read in all information of simRun 2 -- ###
generation = 4
spacer = 4
numMolecules = 12
graftedChainLength = 4
currentSimRun = "codmuc_micelle_g{}_s{}_m{}_l{}_e0.8_b128".format(generation,spacer,numMolecules,graftedChainLength)
filepath = "bfm_files/{}_2225000000_lastconfig.bfm".format(currentSimRun)
# use simulation project function:
insertCodmucSimRunInGraph(myNeoPolymers, currentProject, currentSimRun, filepath, generation, spacer, graftedChainLength)
# add radius of gyration results
filepath = "bfm_files/{}_rg.dat".format(currentSimRun)
myNeoPolymers.addCODMUCRgTensorFileToDatabase(currentSimRun,filepath)


reached mcs command, stop reading. last mcs=  1450000000
path DESKTOP-7NTPDAA: C:/Users/wenge/Documents/Doktorarbeit/neo4j/neo4BFMPolymers/bfm_files/codmuc_micelle_g3_s4_m27_l4_e0.8_b128_rg.dat is added to codmuc_micelle_g3_s4_m27_l4_e0.8_b128 in property named path_1
reached mcs command, stop reading. last mcs=  2250000000
LeMonADEFeature with name FeatureBox already connected to Parameter with name BoxSize and value [128, 128, 128]
LeMonADEFeature with name FeatureBox already connected to Parameter with name Periodicity and value [1, 1, 1]
LeMonADEFeature with name FeatureNNInteractionSc already connected to Parameter with name NNInteraction and value 0.800
path DESKTOP-7NTPDAA: C:/Users/wenge/Documents/Doktorarbeit/neo4j/neo4BFMPolymers/bfm_files/codmuc_micelle_g4_s4_m12_l4_e0.8_b128_rg.dat is added to codmuc_micelle_g4_s4_m12_l4_e0.8_b128 in property named path_1


True

In [4]:
# try ring opening polymerisation bfm files


# this ALWAYS deletes the full graph! Use this function with care
graph.delete_all()

# setup instance of neo4j - BFM file interface
myTestNeoPolymers = neo2BFM.neo2BFMPolymer(graph)

### -- preparation of project -- ###
# add simulation project:
currentProject = "Ring-Opening Polymerisation"
myTestNeoPolymers.addNewSimulationProject(currentProject)
# add ring polymer as polymer:
myTestNeoPolymers.addNewPolymer("Ring Polymer")
# connect Ring Polymer:
myTestNeoPolymers.connectSimulationToPolymer("Ring Polymer", currentProject)

### -- read in all information of simRun 1 -- ###
#RingMelt_N128_Phi05_Box128_EA5_EB17_MCS0_GPU
ring_size = 128
volume_fraction = "05"
activation_energy = 5
breaking_energy = 17

currentSimRun = "RingMelt_N{}_Phi{}_Box128_EA{}_EB{}_MCS0_GPU".format(ring_size,volume_fraction,activation_energy,breaking_energy)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myTestNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myTestNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)

reached mcs command, stop reading. last mcs=  0
> c:\users\wenge\documents\doktorarbeit\neo4j\neo4bfmpolymers\neo2bfmpolymer.py(906)addBFMFileToDatabase()
-> if (featureName in featureList):
(Pdb) n
> c:\users\wenge\documents\doktorarbeit\neo4j\neo4bfmpolymers\neo2bfmpolymer.py(907)addBFMFileToDatabase()
-> numOfRingsKey = "number_of_rings"
(Pdb) n
> c:\users\wenge\documents\doktorarbeit\neo4j\neo4bfmpolymers\neo2bfmpolymer.py(908)addBFMFileToDatabase()
-> numOfRings = self._findElementInKeyValueDataList(numOfRingsKey, dataArray)
(Pdb) numOfRings
*** NameError: name 'numOfRings' is not defined
(Pdb) n
> c:\users\wenge\documents\doktorarbeit\neo4j\neo4bfmpolymers\neo2bfmpolymer.py(909)addBFMFileToDatabase()
-> if (numOfRings is not None):
(Pdb) numOfRings
['1024']
(Pdb) n
> c:\users\wenge\documents\doktorarbeit\neo4j\neo4bfmpolymers\neo2bfmpolymer.py(910)addBFMFileToDatabase()
-> parameterName = "NumberOfRings"
(Pdb) n
> c:\users\wenge\documents\doktorarbeit\neo4j\neo4bfmpolymers\neo2bf

True

In [ ]:
# test some functions

# this ALWAYS deletes the full graph! Use this function with care
graph.delete_all()

myNeoPolymers = neo2BFMPolymer(graph)
print("add linear chain as polymer: ", myNeoPolymers.addNewPolymer("Linear Chain"))
print("add dendrimers as polymer: ", myNeoPolymers.addNewPolymer("Dendrimer"))
print("add star polymer as polymer: ", myNeoPolymers.addNewPolymer("Star Polymer"))
print("add comb polymer as polymer: ", myNeoPolymers.addNewPolymer("Comb Polymer"))

print("add Linear Polymer Melt as SimulationProject: ", myNeoPolymers.addNewSimulationProject("Linear Polymer Melt"))
print("add Single Linear Chain as SimulationProject: ", myNeoPolymers.addNewSimulationProject("Single Linear Chain"))
print("add Single Dendrimer as SimulationProject: ", myNeoPolymers.addNewSimulationProject("Single Dendrimer"))
print("add Mixture Dendrimer and Linear Chains as Simulation: ", myNeoPolymers.addNewSimulationProject("Mixture Dendrimer and Linear Chains"))

print("connect polymer and Melt", myNeoPolymers.connectSimulationToPolymer("Linear Chain", "Linear Polymer Melt"))
print("connect polymer and single chain", myNeoPolymers.connectSimulationToPolymer("Linear Chain", "Single Linear Chain"))
print("connect dendrimers", myNeoPolymers.connectSimulationToPolymer("Dendrimer", "Single Dendrimer"))
print("connect dendrimers", myNeoPolymers.connectSimulationToPolymer("Dendrimer", "Mixture Dendrimer and Linear Chains"))
print("connect dendrimers", myNeoPolymers.connectSimulationToPolymer("Linear Chain", "Mixture Dendrimer and Linear Chains"),"\n")

print("add SimulationRun single dendrimers run 1", myNeoPolymers.addSimulationRunToSimulationProject("Single Dendrimers Solvent Quality","Single Dendrimer"))
print("connect BoxSize", myNeoPolymers.addBoxSizeToSimulationRun("Single Dendrimers Solvent Quality", 64, 64, 64))
print("connect Periodicity", myNeoPolymers.addPeriodicityToSimulationRun("Single Dendrimers Solvent Quality", True, True, True))
print("connect DendrimerGeneration", myNeoPolymers.addDendrimerGenerationToSimulationRun("Single Dendrimers Solvent Quality", 5))
print("connect DendrimerSpacerLength", myNeoPolymers.addDendrimerSpacerLengthToSimulationRun("Single Dendrimers Solvent Quality", 2))
print("connect NNInteraction", myNeoPolymers.addNNInteractionToSimulationRun("Single Dendrimers Solvent Quality", 0.8),"\n")

print("add SimulationRun single chains run 1", myNeoPolymers.addSimulationRunToSimulationProject("Single Linear Chain At Wall","Single Linear Chain"))
print("connect BoxSize", myNeoPolymers.addBoxSizeToSimulationRun("Single Linear Chain At Wall", 64, 64, 64))
print("connect Periodicity", myNeoPolymers.addPeriodicityToSimulationRun("Single Linear Chain At Wall", True, True, False))
print("connect ChainLength", myNeoPolymers.addLinearChainLengthToSimulationRun("Single Linear Chain At Wall", 128))
print("add Result node Rg",myNeoPolymers.addResultRadiusOfGyration("Single Linear Chain At Wall",46.8),"\n")